In [42]:
from glob import glob
import mne
import numpy as np
import pandas as pd
from mne.viz import plot_alignment, set_3d_view


## Reading edf files

In [21]:
all_file_path = glob('data/*.edf')
file_paths = glob('data/seizures/*.seizures')
print(len(all_file_path))
print(len(file_paths))

4
2


In [54]:
raw = mne.io.read_raw_edf(all_file_path[0])
print(raw.get_channel_types())
print(raw.info)
print(raw.ch_names)
print(raw.get_data().shape)

Extracting EDF parameters from /home/guisoares/soares_repo/SCC0276-Machine-Learning/data/chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg']
<Info | 7 non-empty values
 bads: []
 ch_names: FP1-F7, F7-T7, T7-P7, P7-O1, FP1-F3, F3-C3, C3-P3, P3-O1, ...
 chs: 23 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2076-11-06 13:43:04 UTC
 nchan: 23
 projs: []
 sfreq: 256.0 Hz
>
['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']


/tmp/ipykernel_26167/2924192598.py:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(all_file_path[0])


(23, 921600)


In [48]:
def read_data_edf(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5, h_freq=45)
    epochs=mne.make_fixed_length_epochs(data, duration=10, overlap=1)
    array=epochs.get_data()
    return array

# def read_data_seizures(file_path):
#     ref_file = open(file_path, 'r')
#     array=epochs.get_data()
#     return array

In [8]:
%%capture
data_epochs = [read_data_edf(i) for i in all_file_path]


In [17]:
data_epochs = np.array(data_epochs)
data_epochs.shape

(4, 399, 23, 2560)

In [ ]:
# merge the files on one array 
# shape: (N_files, 399 epochs, 23 channels, 2560 values) ->  (N_files*399 epochs, 23 channels, 2560 values)
data_array = np.vstack(data_epochs)
data_array.shape

## Extract features

In [51]:
########! Features extraction !#####
from scipy import stats
def mean(x):
    return np.mean(x, axis=-1)
def std(x):
    return np.std(x, axis=-1)
def ptp(x):
    return np.ptp(x, axis=-1)
def var(x):
    return np.var(x, axis=-1)
def minim(x):
    return np.min(x, axis=-1)
def maxim(x):
    return np.max(x, axis=-1)

def argminim(x):
    return np.argmin(x, axis=-1)

def argmaxim(x):
    return np.argmax(x, axis=-1)

def rms(x):
    return np.sqrt(np.mean(x**2, axis=-1))

def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1)

def skewness(x):
    return stats.skew(x, axis=-1)

def kurtosis(x):
    return stats.kurtosis(x, axis=-1)

def concatenate_features(x):
    return np.concatenate((mean(x), std(x), ptp(x), var(x), minim(x), maxim(x), argmaxim(x), 
                           argminim(x), rms(x), abs_diff_signal(x), skewness(x), kurtosis(x)), axis=-1)

In [52]:
features = []
for d in data_array:
    features.append(concatenate_features(d))
    
features_array = np.array(features)
features_array.shape

(1596, 276)

## Visualize features per channel and per epoch

In [63]:
# get features of specific channel
def features_by_channel(features_array):
    features_channel = []
    MAX_CHANNELS = 23
    for i in range(MAX_CHANNELS):
        array = features_array[:, i*12:(i+1)*12];
        features_channel.append(pd.DataFrame(array, columns=['Mean', 'Std', 'Ptp', 'Var', 'Minimo', 'Maxim', 'Arg Max', 'Arg Min', 'RMS', 'ABS DIFF', 'skewness', 'kurtosis']))
    return features_channel

features_channel = features_by_channel(features_array)
print('Info for channel 23: ')

# Shape of features_channel=(23, 1596, 12)
features_channel[22].shape

Info for channel 23: 


(1596, 12)

## Labelling